In [1]:
!wget -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"

--2019-05-12 01:02:31--  https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.139.181
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.139.181|:443... connected.
HTTP request sent, awaiting response... 416 Requested Range Not Satisfiable

    The file is already fully retrieved; nothing to do.



In [13]:
import requests
import json
import nltk
from nltk.corpus import state_union
from nltk.tokenize import PunktSentenceTokenizer
import numpy as np
import re # 텍스트 전처리
import csv, random
from gensim.models import KeyedVectors # 미리 훈련된 단어 벡터 읽기
from moviepy.editor import *
import io
import base64
from IPython.display import HTML
import io

In [14]:
FILENAME = "GoogleNews-vectors-negative300.bin.gz"
w2v_mod = KeyedVectors.load_word2vec_format(FILENAME, binary=True, limit=300000)

In [4]:
def translate(input_txt, source="ko", target="en"):
    cloud_url = "https://naveropenapi.apigw.ntruss.com/nmt/v1/translation"
    r = requests.post(cloud_url,
                      headers={"X-NCP-APIGW-API-KEY-ID":"ikoxgv3qyf",
                               "X-NCP-APIGW-API-KEY":"SfjIg59wfjz43peNO3yLy0XO152qJ7EU4uEGZTAX"},
                      data={"source": source, "target": target, "text": input_txt})
    info = json.loads(r.text)
    return info["message"]["result"]["translatedText"]

def getKeywords(sentence):
    # input = 'hello I ate chicken'
    # output = ['ate', 'chicken']
    speechs = ['JJ','JJR','JJS','RB','RBR','UH','VB','VBD','VBG','VBN','VBP','VBZ','NN','NNS','NNP','NNPS']
    txt = translate(sentence)
    
    sentences = nltk.sent_tokenize(txt)
    data, datas = [], []
    
    for sent in sentences:
        data = data + nltk.pos_tag(nltk.word_tokenize(sent))
    # print(data)
    for word in data:
        if speechs.count(word[1])!= 0:
            # print(word)
            datas.append(word[0])
    # print(datas)
    return datas

In [20]:
# TODO get keywords
# 변수 설명
# idx: word에 해당하는 index에 대해서  motion 번호를 입력해둠
# word: tag 단어들
# path: 각 모션의 path, 추후 api 사용하게 될 시에 필요함

# keywords = ['play', 'cellphone', 'game', 'teammates', 'knowing']
def most_relevant_tag(keywords):
    f = io.open('data.csv', 'r', encoding='utf-8')
    rdr = csv.reader(f)

    words = []
    path = []
    idx = []

    for i, line in enumerate(rdr):
        n = 0
        tmp = line[2:]
        for w in tmp:
            try:
                w2v_mod[w]
            except KeyError:
                pass
            else:
                if w not in words and w != '':
                    words.append(w)
                    idx.append([i])
                    n = n + 1
                elif w != '':
                    idx[words.index(w)].append(i)
        path.append(line[1])

    f.close()


    s_temp = []

    for k, word in enumerate(words):
        s_temp.append([])
        for j, key in enumerate(keywords):
            s_temp[k].append(w2v_mod.similarity(word, key))

    S = np.sum(s_temp, axis=1)/len(keywords) # 모든 keyword에 대해서 유사도를 합한 행렬

    score = [0] * (i+1) # 각 영상들의 tag 점수를 더해서 tag 갯수로 나눈것
    num = [0] * (i+1) # 영상들의 tag 갯수

    for key, p in enumerate(idx):
        for q in p:
            score[q] = score[q] + S[key]
            num[q] = num[q] + 1

    score_norm = [0]*len(score) # 영상들의 tag의 평균 점수

    for r in range(len(score)):
        score_norm[r] = score[r]/num[r]

    m = np.argmax(score_norm) # max index (can know which tag is the max)

    # 해당 index에 해당하는 모션이 많을 경우 랜덤으로 하나 뽑아줌
    # t = random.sample(idx[m],1)

    # 해당 모션에 대해서 path를 받아옴
    # p = path[t[0]]
    p = path[m]
    return str(m)  # return index
    '''
    f = io.open('data.csv', mode='r', encoding='utf-8')
    rdr = csv.reader(f)

    words = []
    path = []
    idx = []

    for i, line in enumerate(rdr):
        n = 0
        tmp = line[2:]
        for w in tmp:
            try:
                w2v_mod[w]
            except KeyError:
                pass
            else:
                if w not in words and w != '':
                    words.append(w)
                    idx.append([i])
                    n = n + 1
                elif w != '':
                    idx[words.index(w)].append(i)
        path.append(line[1])

    f.close

    W = []

    for word in words:
        try:
            W.append(w2v_mod[word])
        except KeyError:
            pass
    W_n = np.array(W) # tag에 있는 모든 단어들에 대해서 300차원의 vector를 모아둔 n * 300 차원의 행렬

    K = []

    for key in keywords:
        K.append(w2v_mod[key])

    K_n = np.array(K) # 문장에서 얻어낸 keyword의 vector를 모아둔 k* 300 차원의 행렬
    K_n = K_n.transpose() # W와 곱하기 위해서 transpose 해준 300 * k 행렬

    s_temp = np.matmul(W_n , K_n) # keyword에 대해 각각 유사도 점수를 나타내는 행렬

    S = np.sum(s_temp, axis=1)/len(keywords) # 모든 keyword에 대해서 유사도를 합한 행렬

    score = [0] * (i+1) # 각 영상들의 tag 점수를 더해서 tag 갯수로 나눈것
    num = [0] * (i+1) # 영상들의 tag 갯수

    for key, p in enumerate(idx):
        for q in p:
            score[q] = score[q] + S[key]
            num[q] = num[q] + 1

    score_norm = [0]*len(score) # 영상들의 tag의 평균 점수

    for r in range(len(score)):
        score_norm[r] = score[r]/num[r]

    m = np.argmax(score_norm) # max index (can know which tag is the max)

    # 해당 index에 해당하는 모션이 많을 경우 랜덤으로 하나 뽑아줌
    # t = random.sample(idx[m],1)

    # 해당 모션에 대해서 path를 받아옴
    # p = path[t[0]]
    p = path[m]
    return str(m)  # return index
    '''

In [16]:
def produceVideo(clip_list): 
    # input format = ['ANI_74', 'ANI_67', ....]
    # assume that clips are saved locally as ANI_74.MOV, ANI_67.MOV, ...
    if not clip_list:
        print("NO CLIPS")
        return None
    for i in range(len(clip_list)):
        clip_list[i] = clip_list[i] + ".mp4"
    # print("clip_list = %s" % clip_list)
    clip = clip_list[0]
    for i in range(len(clip_list)):
        if i < len(clip_list)-1:
            clip = concatenateVideo(clip, clip_list[i+1])
    return clip
                
def concatenateVideo(path_to_clip1='clip1.mp4', path_to_clip2='clip2.mp4', path_to_output='render.mp4'):
    # clip1 + clip2 => output
    clip1 = VideoFileClip(path_to_clip1)
    clip2 = VideoFileClip(path_to_clip2)
    finalrender = concatenate_videoclips([clip1, clip2])
    finalrender.write_videofile(path_to_output, codec='libx264')
    return path_to_output

def showVideo(path_to_video='render.mp4'):
    video = io.open(path_to_video, 'r+b').read()
    encoded = base64.b64encode(video)
    HTML(data='''<video alt="test" controls>
    <source src="data:video/mp4;base64,{0}" type="video/mp4" />
    </video>'''.format(encoded.decode('ascii')))

In [23]:
if __name__ == "__main__":
    clips = []
    f = open("diary.txt", 'r')
    for line in f:
        sentence = line.split('\n')[0]
        print("sentence = %s" % sentence)
        keywords = getKeywords(sentence)  # sentence -> keywords
        print("keywords = %s" % keywords)   
        clipIndex = most_relevant_tag(keywords)  # keywords -> video!
        clips.append(clipIndex)
    f.close()
    
    
    print("clips index = %s" % clips)
    clips = ['clip1', 'clip2']  # clip 되면 
    path_to_resultVideo = produceVideo(clips)
    video = io.open(path_to_resultVideo, 'r+b').read()
    encoded = base64.b64encode(video)
    HTML(data='''<video alt="test" controls>
    <source src="data:video/mp4;base64,{0}" type="video/mp4" />
    </video>'''.format(encoded.decode('ascii')))


    # 행동하면서 낭독까지??? videobooth 
    # 비디오부스를 뭐로 분류할지? 소팅 --> 서칭이 안되니까.
    # 거기서도 라벨링은 수동으로 : 감정은 아직. 기본적 행동만. 현재는. 대분류 - 소분류 고민중.
    # ㅇㄹㅇㄹㅇㄹㅇㄹㅇㄹ: 어떻게 한꺼번에 다운???
    # 디자인은 이렇게 하겠습니다.
    
    

sentence = Today was a very nice day.
keywords = [u'Today', u'was', u'very', u'nice', u'day']
sentence = I won the Dance Festival Award.
keywords = [u'won', u'Dance', u'Festival', u'Award']
sentence = Everyone clapped their hands.
keywords = [u'Everyone', u'clapped', u'hands']
sentence = It was a proud dance.
keywords = [u'was', u'proud', u'dance']
sentence = Bye
keywords = [u'Bye']
clips index = ['38', '72', '69', '72', '39']


t:   1%|          | 6/889 [00:00<00:15, 58.45it/s, now=None]

Moviepy - Building video render.mp4.
Moviepy - Writing video render.mp4



Moviepy - Done !
Moviepy - video ready render.mp4
